# Frente de Pesquisa NLP

O objetivo desse projeto é criar um pequeno sistema com TF-IDF + consine similarity para recomendar filmes com gêneros parecidos

O sistema consta com 2 datasets, um para a listagem de filmes e seus gêneros e outro com suas respectivas avaliações

[Link para o dataset](https://www.kaggle.com/datasets/gargmanas/movierecommenderdataset?resource=download&select=movies.csv)

Informações sobre o dataset `movies.csv`:

- Coluna de ID: Um único id para cada filme
- Coluna de Título: Título para cada filme
- Coluna de gêneros: objeto com gêneros dos filmes

Informações sobre o dataset `ratings.csv`:

- Coluna de UserID: Um único id do usuário que avaliou o filme
- Coluna do MovieID: Um único id para o filme
- Coluna de Rating: Nota geral do filme, que vai de 0 a 5 (float)
- Coluna de Timestamp: Tempo médio dos usuários que avaliaram esse filme



## Explorando os datasets

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importando os datasets

movies_df = pd.read_csv('/content/movies.csv')
ratings_df = pd.read_csv('/content/ratings.csv')


# Verificação do DataFrame 'movies'
print(" Informações do DataFrame 'movies':")
movies_df.info()
print("\n Primeiras 5 linhas do DataFrame 'movies':")
print(movies_df.head())
print("\n Valores nulos no DataFrame 'movies':")
print(movies_df.isnull().sum())
print("\n Contagem de gêneros únicos no DataFrame 'movies':")
# Para 'genres', 'no genres listed' é um valor comum que pode indicar falta de gênero.
print(movies_df['genres'].value_counts()['(no genres listed)'] if '(no genres listed)' in movies_df['genres'].value_counts() else "Nenhum filme com '(no genres listed)'")


# Verificação do DataFrame 'ratings'
print("\n\n Informações do DataFrame 'ratings':")
ratings_df.info()
print("\n Primeiras 5 linhas do DataFrame 'ratings':")
print(ratings_df.head())
print("\n Valores nulos no DataFrame 'ratings':")
print(ratings_df.isnull().sum())
print("\n Estatísticas descritivas da coluna 'rating' no DataFrame 'ratings':")
print(ratings_df['rating'].describe())


 Informações do DataFrame 'movies':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB

 Primeiras 5 linhas do DataFrame 'movies':
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                              

## Análise exploratória

## Criação do sistema

**TF-IDF + Consine Similarity**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF Vectorizer
# Converter os gêneros em uma matriz TF-IDF
tfidf = TfidfVectorizer(token_pattern=r'[a-zA-Z0-9\-\_]+')
tfidf_matrix = tfidf.fit_transform(movies_df['genres'])

print("Shape da Matriz")
print(tfidf_matrix.shape)
print(movies_df['genres'].shape) # Verificando se o número de linhas bate

# Calcular o cosseno de similaridade
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("Shape da matriz após cosseno de similaridade")
print(cosine_sim.shape)


Shape da Matriz
(9742, 22)
(9742,)
Shape da matriz após cosseno de similaridade
(9742, 9742)


**Mapear similaridade com os índices do filme**

In [ ]:
indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()

**Capturar Avaliações Médias de cada filme**

In [ ]:
avarage_ratings = ratings_df.groupby('movieId')['rating'].mean().reset_index()
avarage_ratings.rename(columns={'rating': 'avarage_rating'}, inplace=True)

**Contar Avaliações de cada filme**

In [ ]:
rating_counts = ratings_df.groupby('movieId')['rating'].count().reset_index()
rating_counts.rename(columns={'rating': 'rating_count'}, inplace=True)

**Criando Função de recomendação**

In [ ]:
def recommend_movies(title, cosine_sim_matrix, df, indices_series, num_recommendations=10):
  if title not in indices_series:
    print(f"Filme '{title}' não encontrado no dataset.")
    return pd.DataFrame()

  idx = indices_series[title]
  sim_scores = list(enumerate(cosine_sim_matrix[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  sim_scores = sim_scores[1:num_recommendations+1]
  movie_indices = [i[0] for i in sim_scores]
  similarity_score = [i[1] for i in sim_scores]

  recommended_df = df.iloc[movie_indices].copy()
  recommended_df['similarity_score'] = similarity_score

  recommended_df['similarity_score'] = recommended_df['similarity_score'].round(2)

  return recommended_df[['title', 'genres', 'average_rating', 'rating_count', 'similarity_score']]

## Criando interface Streamlit

In [ ]:
!pip install streamlit pyngrok -q

In [ ]:
import streamlit as st
import os
from pyngrok import ngrok

@st.cache_data
def load_and_process_data():
    # Verifica se os arquivos existem. No Colab, eles estariam na raiz do ambiente de execução.
    if not os.path.exists('/content/movies.csv') or not os.path.exists('/content/ratings.csv'):
        st.error("Erro: Os arquivos 'movies.csv' ou 'ratings.csv' não foram encontrados. Por favor, certifique-se de tê-los carregado para o ambiente do Colab.")
        st.stop()
    try:
        movies_df = pd.read_csv('/content/movies.csv')
        ratings_df = pd.read_csv('/content/ratings.csv')
    except Exception as e:
        st.error(f"Erro ao carregar os arquivos CSV: {e}")
        st.stop()

    movies_df['genres'] = movies_df['genres'].replace('(no genres listed)', '')

    average_ratings = ratings_df.groupby('movieId')['rating'].mean().reset_index()
    average_ratings.rename(columns={'rating': 'average_rating'}, inplace=True)
    rating_counts = ratings_df.groupby('movieId')['rating'].count().reset_index()
    rating_counts.rename(columns={'rating': 'rating_count'}, inplace=True)

    movies_df = pd.merge(movies_df, average_ratings, on='movieId', how='left')
    movies_df = pd.merge(movies_df, rating_counts, on='movieId', how='left')

    tfidf = TfidfVectorizer(token_pattern=r'[a-zA-Z0-9\-\_]+')
    tfidf_matrix = tfidf.fit_transform(movies_df['genres'])

    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()

    return movies_df, cosine_sim, indices

def recommend_movies(title, cosine_sim_matrix, df, indices_series, num_recommendations=10):
    if title not in indices_series:
        st.warning(f"Filme '{title}' não encontrado no dataset. Por favor, verifique a ortografia ou tente outro filme.")
        return pd.DataFrame()

    idx = indices_series[title]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:num_recommendations+1]
    movie_indices = [i[0] for i in sim_scores]
    similarity_scores = [i[1] for i in sim_scores]

    recommended_df = df.iloc[movie_indices].copy()
    recommended_df['similarity_score'] = similarity_scores
    recommended_df['similarity_score'] = recommended_df['similarity_score'].round(2)

    return recommended_df[['title', 'genres', 'average_rating', 'rating_count', 'similarity_score']]

# Salvar o código do Streamlit em um arquivo .py
streamlit_script_name = "streamlit_app.py"
with open(streamlit_script_name, "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

# --- Lógica principal do app Streamlit ---
# st.set_page_config DEVE SER A PRIMEIRA CHAMADA STREAMLIT
st.set_page_config(page_title="Sistema de Recomendação de Filmes", layout="wide")

# --- Funções e carregamento de dados (copiadas do script principal) ---

@st.cache_data
def load_and_process_data():
    if not os.path.exists('movies.csv') or not os.path.exists('ratings.csv'):
        st.error("Erro: Os arquivos 'movies.csv' ou 'ratings.csv' não foram encontrados. Por favor, certifique-se de tê-los carregado para o ambiente do Colab.")
        st.stop()

    try:
        movies_df = pd.read_csv('movies.csv')
        ratings_df = pd.read_csv('ratings.csv')
    except Exception as e:
        st.error(f"Erro ao carregar os arquivos CSV: {e}")
        st.stop()

    movies_df['genres'] = movies_df['genres'].replace('(no genres listed)', '')
    average_ratings = ratings_df.groupby('movieId')['rating'].mean().reset_index()
    average_ratings.rename(columns={'rating': 'average_rating'}, inplace=True)
    rating_counts = ratings_df.groupby('movieId')['rating'].count().reset_index()
    rating_counts.rename(columns={'rating': 'rating_count'}, inplace=True)
    movies_df = pd.merge(movies_df, average_ratings, on='movieId', how='left')
    movies_df = pd.merge(movies_df, rating_counts, on='movieId', how='left')
    movies_df['average_rating'].fillna(0, inplace=True)
    movies_df['rating_count'].fillna(0, inplace=True)

    tfidf = TfidfVectorizer(token_pattern=r'[a-zA-Z0-9\-\_]+')
    tfidf_matrix = tfidf.fit_transform(movies_df['genres'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()

    return movies_df, cosine_sim, indices

def recommend_movies(title, cosine_sim_matrix, df, indices_series, num_recommendations=10):
    if title not in indices_series:
        st.warning(f"Filme '{title}' não encontrado no dataset. Por favor, verifique a ortografia ou tente outro filme.")
        return pd.DataFrame()

    idx = indices_series[title]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations+1]
    movie_indices = [i[0] for i in sim_scores]
    similarity_scores = [i[1] for i in sim_scores]
    recommended_df = df.iloc[movie_indices].copy()
    recommended_df['similarity_score'] = similarity_scores
    recommended_df['similarity_score'] = recommended_df['similarity_score'].round(2)
    return recommended_df[['title', 'genres', 'average_rating', 'rating_count', 'similarity_score']]

# Carregar dados e modelos uma vez, globalmente para o app Streamlit
movies_df_global, cosine_sim_matrix_global, indices_global = load_and_process_data()

st.title("Sistema de Recomendação de Filmes por Gênero")
st.markdown("Este sistema recomenda filmes com base na similaridade de seus gêneros, utilizando TF-IDF e Similaridade de Cosseno.")

# Slider para o número de recomendações
num_recommendations = st.slider(
    "Quantos filmes você gostaria de recomendar?",
    min_value=5,
    max_value=20,
    value=10,
    step=1
)

# Campo de entrada para o usuário digitar o nome do filme
movie_search_query = st.text_input("Digite o nome de um filme para obter recomendações:", "Toy Story (1995)")

# Lógica para sugestões de filmes
filtered_movies = []
if movie_search_query:
    # Filtra filmes que contêm o texto digitado (case-insensitive)
    filtered_movies = movies_df_global[
        movies_df_global['title'].str.contains(movie_search_query, case=False, na=False)
    ]['title'].tolist()
    # Limita o número de sugestões para não sobrecarregar a interface
    filtered_movies = sorted(filtered_movies)[:50] # Pega até 50 sugestões e ordena alfabeticamente

selected_movie = None
if filtered_movies:
    st.markdown("---")
    st.write("Sugestões (selecione o filme exato):")
    # Tenta pré-selecionar o filme que o usuário digitou se houver uma correspondência exata
    try:
        default_index = filtered_movies.index(movie_search_query)
    except ValueError:
        default_index = 0 # Seleciona o primeiro da lista se não houver correspondência exata

    selected_movie = st.selectbox(
        "Selecione o filme desejado da lista de sugestões:",
        options=filtered_movies,
        index=default_index,
        key="movie_selection_box" # Uma chave única é necessária para widgets
    )
    # Se um filme foi selecionado do selectbox, use-o para a recomendação
    movie_to_recommend = selected_movie
else:
    # Se não houver sugestões, ou se o campo de busca estiver vazio, use o que foi digitado
    movie_to_recommend = movie_search_query

# Botão para acionar a recomendação
if st.button("Obter Recomendações"):
    if movie_to_recommend: # Verifica se há um filme para recomendar
        with st.spinner(f'Buscando {num_recommendations} recomendações para "{movie_to_recommend}"...'):
            recommendations = recommend_movies(movie_to_recommend, cosine_sim_matrix_global, movies_df_global, indices_global, num_recommendations)

            if not recommendations.empty:
                st.subheader(f"Filmes recomendados para '{movie_to_recommend}':")
                st.dataframe(recommendations)
            else:
                st.info("Nenhuma recomendação encontrada ou o filme não existe. Tente outro nome.")
    else:
        st.warning("Por favor, digite o nome de um filme ou selecione um nas sugestões.")

st.markdown("---")
st.write("Desenvolvido com Streamlit, TF-IDF e Cosine Similarity.")
""")

# Iniciar o túnel ngrok (Mantenha o seu token aqui)
ngrok.set_auth_token("{CHAVE_API_NGROK}") # Substitua pelo seu token

print("Aguarde, iniciando o Streamlit e o túnel ngrok...")
public_url = ngrok.connect(8501)
print(f"Seu aplicativo Streamlit está disponível em: {public_url}")

# Rodar o Streamlit em segundo plano
!nohup streamlit run {streamlit_script_name} >/dev/null 2>&1 &

2025-06-16 19:33:02.532 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


Aguarde, iniciando o Streamlit e o túnel ngrok...
Seu aplicativo Streamlit está disponível em: NgrokTunnel: "https://d0a1-35-199-0-234.ngrok-free.app" -> "http://localhost:8501"

Se o link não aparecer, tente reiniciar a célula.
Lembre-se de fazer o upload de movies.csv e ratings.csv antes de rodar este código.
